# 1. Install libraries FIRST


In [9]:
!pip install --quiet yfinance statsmodels backtrader

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
import itertools
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import backtrader as bt
import matplotlib.pyplot as plt

from statsmodels.tsa.stattools import coint
from IPython.display import display

# ---- Base project directory ----
BASE_DIR = "/content/drive/MyDrive/stat_arb_pairs"

DATA_DIR_RAW = os.path.join(BASE_DIR, "data", "raw")
DATA_DIR_CLEANED = os.path.join(BASE_DIR, "data", "cleaned")
PAIRS_DIR = os.path.join(BASE_DIR, "data", "pairs")
REPORT_DIR = os.path.join(BASE_DIR, "reports")

for d in [DATA_DIR_RAW, DATA_DIR_CLEANED, PAIRS_DIR, REPORT_DIR]:
    os.makedirs(d, exist_ok=True)

print("BASE_DIR:", BASE_DIR)
print("CLEANED:", DATA_DIR_CLEANED)
print("PAIRS:", PAIRS_DIR)
print("REPORTS:", REPORT_DIR)


Mounted at /content/drive
BASE_DIR: /content/drive/MyDrive/stat_arb_pairs
CLEANED: /content/drive/MyDrive/stat_arb_pairs/data/cleaned
PAIRS: /content/drive/MyDrive/stat_arb_pairs/data/pairs
REPORTS: /content/drive/MyDrive/stat_arb_pairs/reports


# 2. Load cleaned prices & cointegrated pairs

In [10]:
prices_path = os.path.join(DATA_DIR_CLEANED, "adj_close_wide.csv")
coint_pairs_path = os.path.join(PAIRS_DIR, "cointegrated_pairs.csv")

prices = pd.read_csv(prices_path, parse_dates=["date"], index_col="date")
cointegrated_pairs = pd.read_csv(coint_pairs_path)

print("Prices shape:", prices.shape)
print("Cointegrated pairs:", len(cointegrated_pairs))
display(cointegrated_pairs.head())


# Helper: select top non-overlapping pairs
def select_top_non_overlapping_pairs(coint_df, max_pairs=3, sort_by="pvalue"):
    if sort_by == "pvalue":
        df = coint_df.sort_values("pvalue", ascending=True).copy()
    elif sort_by == "corr":
        df = coint_df.sort_values("corr", ascending=False).copy()
    else:
        df = coint_df.copy()

    used_tickers = set()
    selected_rows = []

    for _, row in df.iterrows():
        a1, a2 = row["asset_1"], row["asset_2"]
        if a1 in used_tickers or a2 in used_tickers:
            continue
        selected_rows.append(row)
        used_tickers.add(a1)
        used_tickers.add(a2)
        if len(selected_rows) >= max_pairs:
            break

    return pd.DataFrame(selected_rows)


top_pairs = select_top_non_overlapping_pairs(
    cointegrated_pairs, max_pairs=3, sort_by="pvalue"
)
if top_pairs.empty:
    raise ValueError("No cointegrated pairs selected for optimization. Run 01 notebook and check cointegration results.")


print("Selected pairs for optimization universe:")
display(top_pairs)


# Helper: build OHLC from close-only series
def make_ohlc_from_close(close_series):
    df = pd.DataFrame(index=close_series.index)
    df["close"] = close_series
    df["open"] = close_series
    df["high"] = close_series
    df["low"] = close_series
    df["volume"] = 1_000_000  # dummy volume
    return df


# Build OHLC dict for all unique tickers in top_pairs
tickers = sorted(set(top_pairs["asset_1"]).union(set(top_pairs["asset_2"])))
print("Tickers in optimization portfolio:", tickers)

ohlc_dict = {}
for ticker in tickers:
    series = prices[ticker].dropna()
    ohlc_dict[ticker] = make_ohlc_from_close(series)

# Align on common date range across all tickers
common_index = ohlc_dict[tickers[0]].index
for t in tickers[1:]:
    common_index = common_index.intersection(ohlc_dict[t].index)

for t in tickers:
    ohlc_dict[t] = ohlc_dict[t].loc[common_index]

print("Common date range:", common_index.min(), "to", common_index.max())

# Build pair_list
pair_list = []
for _, row in top_pairs.iterrows():
    pair_list.append(
        {
            "asset_1": row["asset_1"],
            "asset_2": row["asset_2"],
            "hedge_ratio": row["hedge_ratio"],
        }
    )

print("pair_list used for optimization:")
pair_list


Prices shape: (1255, 16)
Cointegrated pairs: 3


,asset_1,asset_2,corr,pvalue,coint_t,crit_1pct,crit_5pct,crit_10pct,hedge_ratio,intercept,n_obs
0,DIA,SPY,0.920666,0.016986,-3.725514,-3.905195,-3.341007,-3.047834,0.569824,88.777271,1255
1,DIA,XLK,0.766556,0.022805,-3.625058,-3.905195,-3.341007,-3.047834,2.219993,156.740707,1255
2,DIA,QQQ,0.780279,0.041631,-3.406521,-3.905195,-3.341007,-3.047834,0.543489,139.857228,1255


Selected pairs for optimization universe:


,asset_1,asset_2,corr,pvalue,coint_t,crit_1pct,crit_5pct,crit_10pct,hedge_ratio,intercept,n_obs
0,DIA,SPY,0.920666,0.016986,-3.725514,-3.905195,-3.341007,-3.047834,0.569824,88.777271,1255


Tickers in optimization portfolio: ['DIA', 'SPY']
Common date range: 2020-12-08 00:00:00 to 2025-12-05 00:00:00
pair_list used for optimization:


[{'asset_1': 'DIA', 'asset_2': 'SPY', 'hedge_ratio': 0.5698243414015355}]

#3. KalmanPairsTrading Strategy

In [11]:
class KalmanPairsTrading(bt.Strategy):
    params = dict(
        pair_list=None,
        window=60,
        entry_z=2.5,
        exit_z=0.5,
        stop_z=4.0,
        stake=50,
        long_only=False,
        delta=1e-5,
        R=0.001,
    )

    def __init__(self):
        if not self.p.pair_list:
            raise ValueError("pair_list must be provided")

        self.data_dict = {d._name: d for d in self.datas}

        self.pairs = []
        for cfg in self.p.pair_list:
            a1 = cfg["asset_1"]
            a2 = cfg["asset_2"]
            hedge_ratio = cfg.get("hedge_ratio", 1.0)

            if a1 not in self.data_dict or a2 not in self.data_dict:
                print(f"Skipping missing pair: {a1}-{a2}")
                continue

            self.pairs.append(
                {
                    "asset_1": a1,
                    "asset_2": a2,
                    "data_A": self.getdatabyname(a1),
                    "data_B": self.getdatabyname(a2),
                    "beta": hedge_ratio,
                    "P": 1.0,
                    "spread_buffer": [],
                    "state": 0,
                }
            )

        print(f"Initialized strategy with {len(self.pairs)} pairs (Kalman hedge).")

    def next(self):
        for pair in self.pairs:
            data_A = pair["data_A"]
            data_B = pair["data_B"]

            if len(data_A) == 0 or len(data_B) == 0:
                continue

            price_A = data_A.close[0]
            price_B = data_B.close[0]

            # === Kalman update ===
            beta = pair["beta"]
            P = pair["P"]

            Q = self.p.delta / (1 - self.p.delta)
            beta_prior = beta
            P_prior = P + Q

            if price_B != 0:
                S = P_prior * price_B**2 + self.p.R
                K = (P_prior * price_B) / S
                y = price_A - beta_prior * price_B

                beta = beta_prior + K * y
                P = (1 - K * price_B) * P_prior

            pair["beta"] = beta
            pair["P"] = P

            # === Spread + Z-score ===
            spread = price_A - beta * price_B
            pair["spread_buffer"].append(spread)

            if len(pair["spread_buffer"]) < self.p.window:
                continue

            buf = pair["spread_buffer"][-self.p.window:]
            mean = np.mean(buf)
            std = np.std(buf)
            if std == 0:
                continue

            z = (spread - mean) / std

            state = pair["state"]
            pos_A = self.getposition(data_A).size
            pos_B = self.getposition(data_B).size

            # === ENTRY ===
            if state == 0 and pos_A == 0 and pos_B == 0:
                # Long spread
                if z < -self.p.entry_z:
                    qty_B = self.p.stake
                    qty_A = int(beta * qty_B)
                    self.buy(data_A, size=qty_A)
                    self.sell(data_B, size=qty_B)
                    pair["state"] = 1

                # Short spread
                elif not self.p.long_only and z > self.p.entry_z:
                    qty_B = self.p.stake
                    qty_A = int(beta * qty_B)
                    self.sell(data_A, size=qty_A)
                    self.buy(data_B, size=qty_B)
                    pair["state"] = -1

            # === EXIT (mean reversion or stop loss) ===
            elif state != 0:
                if abs(z) < self.p.exit_z or abs(z) > self.p.stop_z:
                    if pos_A > 0:
                        self.sell(data_A, size=pos_A)
                    elif pos_A < 0:
                        self.buy(data_A, size=abs(pos_A))

                    if pos_B > 0:
                        self.sell(data_B, size=pos_B)
                    elif pos_B < 0:
                        self.buy(data_B, size=abs(pos_B))

                    pair["state"] = 0


# 4. Helper: run one backtest for given hyperparameters

In [12]:
def run_backtest_for_params(
    window,
    entry_z,
    exit_z,
    stop_z,
    stake,
    long_only,
    delta=1e-5,
    R=0.001,
    initial_cash=100000.0,
    commission=0.0005,
):
    cerebro = bt.Cerebro()

    # Rebuild fresh data feeds for this run
    for ticker in tickers:
        df_t = ohlc_dict[ticker].copy()
        df_t = df_t[["open", "high", "low", "close", "volume"]]
        data_feed = bt.feeds.PandasData(dataname=df_t)
        cerebro.adddata(data_feed, name=ticker)

    cerebro.addstrategy(
        KalmanPairsTrading,
        pair_list=pair_list,
        window=window,
        entry_z=entry_z,
        exit_z=exit_z,
        stop_z=stop_z,
        stake=stake,
        long_only=long_only,
        delta=delta,
        R=R,
    )

    cerebro.broker.setcash(initial_cash)
    cerebro.broker.setcommission(commission=commission)

    # Analyzers
    cerebro.addanalyzer(bt.analyzers.TimeReturn, _name="timereturn", timeframe=bt.TimeFrame.Days)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe", timeframe=bt.TimeFrame.Days, riskfreerate=0.0)
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name="dd")
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trades")

    results = cerebro.run()
    strat = results[0]

    # TimeReturn → daily returns
    ret_series = pd.Series(strat.analyzers.timereturn.get_analysis())
    equity_curve = (1 + ret_series).cumprod() * initial_cash

    final_value = equity_curve.iloc[-1] if len(equity_curve) > 0 else initial_cash
    total_return = (final_value / initial_cash) - 1

    # Sharpe
    sharpe_dict = strat.analyzers.sharpe.get_analysis()
    sharpe_ratio = sharpe_dict.get("sharperatio", np.nan)

    # Drawdown
    dd_dict = strat.analyzers.dd.get_analysis()
    max_dd_pct = dd_dict.get("max", {}).get("drawdown", np.nan)
    max_dd_abs = dd_dict.get("max", {}).get("moneydown", np.nan)

    # Trades
    trades_dict = strat.analyzers.trades.get_analysis()
    total_trades = trades_dict.get("total", {}).get("closed", 0)

    metrics = {
        "window": window,
        "entry_z": entry_z,
        "exit_z": exit_z,
        "stop_z": stop_z,
        "stake": stake,
        "long_only": long_only,
        "delta": delta,
        "R": R,
        "final_value": final_value,
        "total_return": total_return,
        "sharpe_ratio": sharpe_ratio,
        "max_dd_pct": max_dd_pct,
        "max_dd_abs": max_dd_abs,
        "total_trades": total_trades,
        "n_days": len(ret_series),
    }

    return metrics


# 5. Define parameter grid & run optimization

In [13]:
import itertools

# Reasonable small grid (can expand later)
WINDOW_LIST = [45, 60, 90]
ENTRY_Z_LIST = [2.0, 2.5]
EXIT_Z_LIST = [0.5]
STOP_Z_LIST = [3.5, 4.0]
STAKE_LIST = [25, 50, 75]
LONG_ONLY_LIST = [False, True]

param_grid = list(
    itertools.product(
        WINDOW_LIST,
        ENTRY_Z_LIST,
        EXIT_Z_LIST,
        STOP_Z_LIST,
        STAKE_LIST,
        LONG_ONLY_LIST,
    )
)

print("Total combinations:", len(param_grid))

results_list = []

for (
    window,
    entry_z,
    exit_z,
    stop_z,
    stake,
    long_only,
) in param_grid:
    print(
        f"Running: window={window}, entry_z={entry_z}, exit_z={exit_z}, "
        f"stop_z={stop_z}, stake={stake}, long_only={long_only}"
    )
    try:
        metrics = run_backtest_for_params(
            window=window,
            entry_z=entry_z,
            exit_z=exit_z,
            stop_z=stop_z,
            stake=stake,
            long_only=long_only,
        )
        results_list.append(metrics)
    except Exception as e:
        print("Error with config, skipping:", e)
        continue

opt_df = pd.DataFrame(results_list)
print("Optimization runs completed:", len(opt_df))
opt_df.head()


Total combinations: 72
Running: window=45, entry_z=2.0, exit_z=0.5, stop_z=3.5, stake=25, long_only=False
Initialized strategy with 1 pairs (Kalman hedge).
Running: window=45, entry_z=2.0, exit_z=0.5, stop_z=3.5, stake=25, long_only=True
Initialized strategy with 1 pairs (Kalman hedge).
Running: window=45, entry_z=2.0, exit_z=0.5, stop_z=3.5, stake=50, long_only=False
Initialized strategy with 1 pairs (Kalman hedge).
Running: window=45, entry_z=2.0, exit_z=0.5, stop_z=3.5, stake=50, long_only=True
Initialized strategy with 1 pairs (Kalman hedge).
Running: window=45, entry_z=2.0, exit_z=0.5, stop_z=3.5, stake=75, long_only=False
Initialized strategy with 1 pairs (Kalman hedge).
Running: window=45, entry_z=2.0, exit_z=0.5, stop_z=3.5, stake=75, long_only=True
Initialized strategy with 1 pairs (Kalman hedge).
Running: window=45, entry_z=2.0, exit_z=0.5, stop_z=4.0, stake=25, long_only=False
Initialized strategy with 1 pairs (Kalman hedge).
Running: window=45, entry_z=2.0, exit_z=0.5, stop

,window,entry_z,exit_z,stop_z,stake,long_only,delta,R,final_value,total_return,sharpe_ratio,max_dd_pct,max_dd_abs,total_trades,n_days
0,45,2.0,0.5,3.5,25,False,0.00001,0.001,99927.122239,-0.000729,-0.001497,0.945169,953.492214,106,1255
1,45,2.0,0.5,3.5,25,True,0.00001,0.001,98803.700211,-0.011963,-0.034035,1.382834,1385.449489,46,1255
2,45,2.0,0.5,3.5,50,False,0.00001,0.001,99819.103407,-0.001809,-0.001767,1.849913,1881.370366,106,1255
3,45,2.0,0.5,3.5,50,True,0.00001,0.001,97637.534650,-0.023625,-0.034064,2.724939,2735.093226,46,1255
4,45,2.0,0.5,3.5,75,False,0.00001,0.001,99695.378217,-0.003046,-0.001875,2.755580,2825.031959,106,1255


# 6. Analyze optimization results & save

In [14]:
if opt_df.empty:
    raise ValueError("No successful optimization runs. Check grid or strategy config.")

# Sorting primarily by Sharpe, secondarily by max drawdown (ascending)
opt_df_sorted = opt_df.sort_values(
    by=["sharpe_ratio", "max_dd_pct"],
    ascending=[False, True],
).reset_index(drop=True)

print("Top 10 configurations by Sharpe:")
display(opt_df_sorted.head(10))

# Saving to reports
opt_results_path = os.path.join(REPORT_DIR, "kalman_param_optimization_results.csv")
opt_df_sorted.to_csv(opt_results_path, index=False)
print("Saved optimization results to:", opt_results_path)


Top 10 configurations by Sharpe:


,window,entry_z,exit_z,stop_z,stake,long_only,delta,R,final_value,total_return,sharpe_ratio,max_dd_pct,max_dd_abs,total_trades,n_days
0,60,2.5,0.5,3.5,25,False,0.00001,0.001,100497.077275,0.004971,0.033068,0.223872,225.489606,34,1255
1,60,2.5,0.5,4.0,25,False,0.00001,0.001,100497.077275,0.004971,0.033068,0.223872,225.489606,34,1255
2,60,2.5,0.5,3.5,75,False,0.00001,0.001,101464.764527,0.014648,0.032920,0.654114,668.065642,34,1255
3,60,2.5,0.5,4.0,75,False,0.00001,0.001,101464.764527,0.014648,0.032920,0.654114,668.065642,34,1255
4,60,2.5,0.5,3.5,50,False,0.00001,0.001,100973.830140,0.009738,0.032689,0.445476,451.826812,34,1255
5,60,2.5,0.5,4.0,50,False,0.00001,0.001,100973.830140,0.009738,0.032689,0.445476,451.826812,34,1255
6,45,2.5,0.5,3.5,75,False,0.00001,0.001,100203.151467,0.002032,0.004249,1.043519,1048.868290,36,1255
7,45,2.5,0.5,4.0,75,False,0.00001,0.001,100203.151467,0.002032,0.004249,1.043519,1048.868290,36,1255
8,45,2.5,0.5,3.5,25,False,0.00001,0.001,100070.294525,0.000703,0.004201,0.357294,357.941039,36,1255
9,45,2.5,0.5,4.0,25,False,0.00001,0.001,100070.294525,0.000703,0.004201,0.357294,357.941039,36,1255


Saved optimization results to: /content/drive/MyDrive/stat_arb_pairs/reports/kalman_param_optimization_results.csv


# 7. Export best configuration as JSON

In [15]:
import json

best_cfg = opt_df_sorted.iloc[0].to_dict()

CONFIG_DIR = os.path.join(BASE_DIR, "config")
os.makedirs(CONFIG_DIR, exist_ok=True)

config_path = os.path.join(CONFIG_DIR, "kalman_prod_config.json")

# Only keep relevant hyperparameters in config
config_to_save = {
    "window": int(best_cfg["window"]),
    "entry_z": float(best_cfg["entry_z"]),
    "exit_z": float(best_cfg["exit_z"]),
    "stop_z": float(best_cfg["stop_z"]),
    "stake": int(best_cfg["stake"]),
    "long_only": bool(best_cfg["long_only"]),
    "delta": float(best_cfg["delta"]),
    "R": float(best_cfg["R"]),
    "initial_cash": 100000.0,
    "commission": 0.0005,
    "note": "Auto-generated from 02_kalman_optimization.ipynb",
}

with open(config_path, "w") as f:
    json.dump(config_to_save, f, indent=4)

print("Best config saved to:", config_path)
config_to_save


Best config saved to: /content/drive/MyDrive/stat_arb_pairs/config/kalman_prod_config.json


{'window': 60,
 'entry_z': 2.5,
 'exit_z': 0.5,
 'stop_z': 3.5,
 'stake': 25,
 'long_only': False,
 'delta': 1e-05,
 'R': 0.001,
 'initial_cash': 100000.0,
 'commission': 0.0005,
 'note': 'Auto-generated from 02_kalman_optimization.ipynb'}